In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://apache.mirror.colo-serv.net/spark/spark-2.4.7/spark-2.4.7-bin-hadoop2.7.tgz
!tar xf spark-2.4.7-bin-hadoop2.7.tgz
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.7-bin-hadoop2.7"

import findspark
findspark.init("spark-2.4.7-bin-hadoop2.7")# SPARK_HOME

import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

sc = SparkContext.getOrCreate()
spark = SparkSession.builder.getOrCreate()



Reading package lists... Done
Building dependency tree       
Reading state information... Done
python3 is already the newest version (3.6.7-1~18.04).
python3 set to manually installed.
0 upgraded, 0 newly installed, 0 to remove and 30 not upgraded.


In [ ]:
# geting the data. Using the code from Q&A
!wget http://files.grouplens.org/datasets/movielens/ml-latest-small.zip
!unzip ml-latest-small.zip
!mv ml-latest-small/ratings.csv .
!mv ml-latest-small/movies.csv .
linesRDD = sc.textFile("ratings.csv")

#1. Find the average rating for each user and movie

In [ ]:
# filter the data to only contain the column user and movie
linesRDD = sc.textFile("ratings.csv")
header=linesRDD.first()
linesRDD=linesRDD.filter(lambda row: True if row != header else False)

#tuples:
lines_list = linesRDD.map(lambda line: line.split(','))

#flatmap, here we actually do not need the last timestamp for calculating the average of each user. 
#we need to get the pair of each (user rating) and (movie rating)
user_rating=lines_list.flatMap(lambda data: (('u'+data[0],float(data[2])),))
movie_rating=lines_list.flatMap(lambda data: (('m'+data[1],float(data[2])),))


#check the output
print(linesRDD.take(5))
print(lines_list.take(5))
print(user_rating.take(10))
print(movie_rating.take(10))

# New Section

In [ ]:
#first input the data to the script
from google.colab import drive
drive.mount('/content/drive')

import csv
datafile = open('/content/movies.csv', 'r')
myreader = csv.reader(datafile)

movieGenres = {}
for row in myreader:
    movieGenres[row[0]] = row[2].split('|')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
type(movieGenres)

dict

In [ ]:
#as we know that the movieGenres are a dictionary of python, we can directy use it to map to the RDD 
#here we map the rating to the genre and match the movie id. 
combined_rdd=lines_list.flatMap(lambda rdd:[(genre,float(rdd[2])) for genre in movieGenres[rdd[1]]])

In [ ]:
#then we the same method as the first question to calcualte the average for each genre
sumcntRDD_combined = combined_rdd.aggregateByKey((0,0),lambda acc,rating: (acc[0]+rating, acc[1]+1), lambda acc1, acc2: (acc1[0]+acc2[0], acc1[1]+acc2[1]))

print(sumcntRDD_combined.take(5))
genre_result =sumcntRDD_combined.mapValues(lambda value: value[0]/value[1])
print("The average rating for each genre is: ")
print(genre_result.take(5))

[('Children', (31426.5, 9208)), ('Fantasy', (41312.5, 11834)), ('Romance', (63552.0, 18124)), ('Action', (105629.0, 30635)), ('Thriller', (92415.5, 26452))]
The average rating for each genre is: 
[('Children', 3.412956125108601), ('Fantasy', 3.4910005070136894), ('Romance', 3.5065107040388437), ('Action', 3.447984331646809), ('Thriller', 3.4937055799183425)]


In [ ]:
A = parallelize([('d1',['cat','tiger','lion','leopard']), \
('d2',['tiger','lion','jaguar']), \
('d3',['dog','wolf','jackal']), \
('d4',['dog','coyotte','jackal','wolf'])])

NameError: ignored